<a href="https://colab.research.google.com/github/hamnakhan11/hamna/blob/main/General_Health_Query_Chatbot_task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade streamlit pyngrok transformers accelerate --quiet --no-deps --ignore-installed blinker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 19.9 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers huggingface-hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 58.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0


In [ ]:
# Streamlit chatbot for general health questions using TinyLlama

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import textwrap # Import textwrap for formatting

# Load model and tokenizer only once
# @st.cache_resource # Removed Streamlit cache decorator
def load_model():
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

# Build prompt with clear instructions
def build_prompt(user_input):
    return f"""[INST] You are a friendly medical assistant. Provide general, safe health information in simple language. Avoid giving any harmful or risky advice, and always remind the user to consult a licensed doctor for serious concerns.

User: {user_input}
Assistant:"""

# Filter out dangerous queries
def is_safe_query(query):
    harmful_keywords = [
        "suicide", "overdose", "kill", "die", "death", "emergency",
        "diagnose", "prescribe", "surgery", "bleeding", "poison", "inject"
    ]
    return not any(word in query.lower() for word in harmful_keywords)

# Get model response
def get_response(pipe, user_input):
    if not is_safe_query(user_input):
        return "⚠️ I'm not able to help with that. Please contact a licensed healthcare professional."
    prompt = build_prompt(user_input)
    result = pipe(prompt, max_new_tokens=512, temperature=0.7, do_sample=True)[0]["generated_text"] # Increased max_new_tokens

    # Remove unwanted token echoes
    if "Assistant:" in result:
        response = result.split("Assistant:")[-1].strip()
        response = response.split("get_response")[0].strip()
        return response
    return result.strip()

# Load the model once
pipe = load_model()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
while True:
    user_input = input("You: ")
    if not user_input:
        break  # Exit the loop if the user enters an empty line
    print("Assistant: ", get_response(pipe, user_input))

You: i am sick
Assistant:  yeah, I'd be happy to. There are several things you can do to ease your pain. First, take some over-the-counter pain relievers, such as ibuprofen or acetaminophen. If those don't work, consider seeing a doctor. Second, try eating a healthy meal, such as a banana or an apple, to help soothe your stomach. Third, drink plenty of fluids, such as water or herbal tea, to help flush out any excess fluids. Fourth, try to relax and avoid stress, as that can exacerbate your pain. Finally, don't hesitate to call your doctor if your pain persists or worsens. Remember, the most important thing is to listen to your body and take care of yourself. Always seek medical advice if you have any concerns or questions.
You: i feel cold and pain
Assistant:  Please wait a few days and call me back if you need an appointment. Seeking medical attention is important for your health.
You: 
